In [3]:
from pathlib import Path
termlistPath = Path("../master-database-files/master-keyword-extraction/oxford_terms.txt")
outputPath = Path("../master-database-files/master-keyword-extraction/most_important_physics_terms.txt")
outputPath.touch()
termlist = termlistPath.read_text().split("\n")

In [7]:
from os import environ
from pathlib import Path
from json import loads, dumps
from random import choice
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()
from openai import OpenAI
from random import randint

openaiClient = OpenAI()
def gpt_3_5_turbo_completion(query, temperature = 1):
    answer = openaiClient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        temperature = temperature,
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def gpt_4_turbo_completion(query, temperature = 1):
    answer = openaiClient.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        temperature = temperature,
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def tryRecieveAnswer(query, completionFunction = gpt_4_turbo_completion, answerConversion = lambda x: True, maxTries = 10, temperature = 1):
    tryNumber = 0
    while tryNumber < maxTries:
        answer = completionFunction(query, temperature)
        try:
            answer = answerConversion(answer)
            return (answer, True)
        except:
            pass
        tryNumber += 1
    print(f"Failed to recieve answer for query: {query}")
    return (None, False)

def listAnswerConversion(answer):
    result = loads(answer)
    assert isinstance(result, list)
    for item in result:
        assert isinstance(item, str)
    return result

In [8]:
def createMostImportantTermsFile():
    with outputPath.open("w") as file:
        start = True
        for i in range(0, len(termlist), 38):
            file.write("\n" if not start else "")
            start = False
            termsSelection = termlist[i:i+38] if i+38 < len(termlist) else termlist[i:]
            numberedTerms = "{" + ", ".join([f'{i+1}: "{term}"' for i, term in enumerate(termsSelection)]) + "}"
            query = f"What is the most important technical physics term in the following list: {numberedTerms}. Return only the number of the term without extra words."
            def answerConversion(answer):
                ret = int(answer)
                assert ret >= 1 and ret <= len(termsSelection)
                return termsSelection[ret-1]
            answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)
            if success:
                file.write(answer)
            file.flush()
        

In [9]:
createMostImportantTermsFile()